# Political Leaders’ Affiliation Database

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [41]:
import pandas as pd
import geopandas as gpd
import altair as alt
from datetime import timedelta
import numpy as np

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [ ]:
# Data: https://dataverse.harvard.edu/file.xhtml?fileId=5211722&version=6.0
# Codebook: https://dataverse.harvard.edu/file.xhtml?fileId=5211721&version=6.0

### Read Excel spreadsheet data

In [43]:
src = pd.read_excel('data/raw/PLAD_Oct_2021.xls')

### Process the dates

In [44]:
src['startdate'] = pd.to_datetime(src['startdate'])
src['enddate'] = pd.to_datetime(src['enddate'])
src['birthdate'] = pd.to_datetime(src['birthdate'], errors='coerce')

In [45]:
src.rename(columns={'adm0':'birth_country', "adm1":"birth_adm1", "adm2":"birth_adm1", "country":"country_led"}, inplace=True)

### Tenure in office

In [56]:
src['days_in_office'] = src['enddate'] - src['startdate']

In [57]:
src['years_in_office'] = (df["days_in_office"] / timedelta(days=365)).round(2)

In [58]:
src['months_in_office'] = ((src['enddate'] - src['startdate']) / np.timedelta64(1, 'M')).round()

### Make a copy of the dataframe for analysis

In [78]:
df = src[['country_led', 'idacr', 'leader', 'gender', 'entry', 'exit', 'startdate', 'enddate', 'days_in_office', 'years_in_office','months_in_office', 'startyear', 'endyear', 'birth_country', 'birth_adm1', 'birth_adm1', 'continent', 'latitude', 'longitude', 'geoname','foreign_leader', 'ethnicity1', 'ethnicity2','ethnicity_geoepr1', 'ethnicity_geoepr2', 'ethnicitysource1','ethnicitysource2', 'ethnicity_precision','yrborn', 'birthdate', 'uid', 'id_0', 'id_1', 'id_2', 'gid_0', 'gid_1','gid_2', 'edu_name', 'edu_r', 'birthplace_comment', 'ethnicity_comment']].copy()

---

In [75]:
still_in_office = df[df['exit'] == 'Still in Office'].sort_values('years_in_office', ascending=False)

In [76]:
still_in_office.head()

,country_led,idacr,leader,gender,entry,exit,startdate,enddate,days_in_office,years_in_office,months_in_office,startyear,endyear,birth_country,birth_adm1,birth_adm1,birth_adm1,birth_adm1,continent,latitude,longitude,geoname,foreign_leader,ethnicity1,ethnicity2,ethnicity_geoepr1,ethnicity_geoepr2,ethnicitysource1,ethnicitysource2,ethnicity_precision,yrborn,birthdate,uid,id_0,id_1,id_2,gid_0,gid_1,gid_2,edu_name,edu_r,birthplace_comment,ethnicity_comment
191,Brunei,BRU,Hassanal Bolkiah,M,Regular,Still in Office,1967-10-04,2020-12-31,19447 days,53.28,639.0,1967,2020,Brunei,Brunei and Muara,Kianggeh,Brunei and Muara,Kianggeh,Asia,4.89035,114.940063,Bandar Seri Begawan,0,Malay,.,Bumiputera (Muslims),.,https://ethnicelebs.com/hassanal-bolkiah,.,2,1946,1946-07-15,263609,36,2,12,BRN,BRN.2_1,BRN.2.4_1,.,6,.,.
367,Equatorial Guinea,EQG,Nguema Mbasogo,M,Irregular,Still in Office,1979-08-03,2020-12-31,15126 days,41.44,497.0,1979,2020,Equatorial Guinea,Wele-Nzas,Mongomo,Wele-Nzas,Mongomo,Africa,1.60994,11.30741,Acoacán,0,Fang,.,Fang,.,https://www.africaresearchinstitute.org/newsite/equatorial-guinea-expert-briefing/,https://www.oxfordreference.com/view/10.1093/oi/authority.20110803100243503,1,1942,1942-06-05,137024,71,7,31,GNQ,GNQ.7_1,GNQ.7.6_1,.,6,.,.
230,Cameroon,CAO,Biya,M,Regular,Still in Office,1982-11-06,2020-12-31,13935 days,38.18,458.0,1982,2020,Cameroon,Sud,Dja et Lobo,Sud,Dja et Lobo,Africa,3.1146,12.2681,Mvomeka'a,0,Beti,.,Beti (and related peoples),.,https://freedomhouse.org/report/freedom-world/2007/cameroon,http://www.allgov.com/news/us-and-the-world/dictator-of-the-month-paul-biya-of-cameroon?news=843343,1,1933,1933-02-13,135810,41,10,55,CMR,CMR.10_1,CMR.10.1_1,.,6,.,.
1212,Uganda,UGA,Museveni,M,Irregular,Still in Office,1986-01-29,2020-12-31,12755 days,34.95,419.0,1986,2020,Uganda,Ntungamo,Ruhaama,Ntungamo,Ruhaama,Africa,-0.935,30.288059,Ntungamo,0,Bahima,.,"South-Westerners (Ankole, Banyoro, Toro)",.,https://www.theguardian.com/society/katineblog/2009/apr/02/museveni-divided-uganda,https://books.google.ge/books?id=T9k0IkER6KYC&pg=PA77&lpg=PA77&dq=museveni+ethnic+ankole&source=bl&ots=0Ue9gPWbLC&sig=ACfU3U1UWv4WUwqQLHL3I8P8BTxjwentHQ&hl=en&sa=X&ved=2ahUKEwjnx7r53bDpAhVF2KQKHSIpAzoQ6AEwAnoECAkQAQ#v=onepage&q=museveni%20ethnic%20ankole&f=false,1,1944,1944-08-15,141763,239,48,136,UGA,UGA.48_1,UGA.48.2_1,.,6,.,.
1109,Eswatini,SWA,Mswati,M,Regular,Still in Office,1986-04-25,2020-12-31,12669 days,34.71,416.0,1986,2020,Swaziland,Manzini,Manzini South,Manzini,Manzini South,Africa,-26.5082,31.371317,Manzini,0,Swazi,.,Swazi,.,https://www.britannica.com/biography/Mswati-III,.,2,1968,1968-04-18,177352,221,3,35,SWZ,SWZ.3_1,SWZ.3.10_1,.,4,.,.
